# Model Training
In this notebook, we will train our VAE model. This involves:

1. Encoder
2. Decoder
3. Full Model

## Importing Packages
We will be using Keras to build and train our VAE

In [18]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib notebook
import tensorflow as tf

import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
import h5py

# import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Add, Multiply, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import model_from_json
# import mdn

from IPython import display
import numpy as np
import pandas as pd
from sklearn import preprocessing
from pickle import dump, load

from matplotlib.ticker import FormatStrFormatter
from IPython.display import SVG

ModuleNotFoundError: No module named 'tensorflow.keras.utils.vis_utils'

## Definition of Constants
Hyper parameters and constants will all be located here for ease of adjustments

In [28]:
categorical_dim = 11
categorical_map = {0:0, 0.5:1, 1: 2, 1.5:3, 2:4, 2.5:5, 3:6, 3.5:7, 4:8, 4.5:9, 5:10}
reverse_categorical_map = {0:0, 1:0.5, 2:1, 3:1.5, 4:2, 5:2.5, 6:3, 7:3.5, 8:4, 9:4.5, 10:5}
continuous_dim = 3
binary_dim = 1
original_dim = binary_dim + continuous_dim + categorical_dim
intermediate_dim_1 = 50
intermediate_dim_2 = 50
latent_dim = 4
batch_size = 100
epochs = 50
epsilon_std = 1.0

## Constants for the Mixture layer
N_HIDDEN = 15  # number of hidden units in the Dense layer
N_MIXES = 10  # number of mixture components
OUTPUT_DIMS = 2  # number of real-values predicted by each mixture component

## Building the Model
We have to write our own custom layer and custom loss function as these are not supported on Keras natively. There are a few things to be done:

1. Custom KLDivergence Layer
2. Custom Loss Functions
3. Building the Model

### KL Divergence Layer
To ensure modularity, we decided to create a separate layer for KL Divergence. This layer will account for the loss required. 

In [3]:
class KLDivergenceLayer(Layer):

    """ Identity transform layer that adds KL divergence
    to the final model loss.
    """

    def __init__(self, *args, **kwargs):
        self.is_placeholder = True
        super(KLDivergenceLayer, self).__init__(*args, **kwargs)

    def call(self, inputs):

        mu, log_var = inputs

        kl_batch = - .5 * K.sum(1 + log_var -
                                K.square(mu) -
                                K.exp(log_var), axis=-1)

        self.add_loss(K.mean(kl_batch), inputs=inputs)

        return inputs

### Custom Loss Functions
As the yelp dataset contains binary, categorical as well as continuous data, we will build 3 custom loss functions.

#### Binary Loss Function

In [4]:
def binary_loss(y_true, y_pred):
	# input dimension is (batchsize, 1)
    return K.binary_crossentropy(y_true, y_pred) # the dimension of return value is (batchsize , 1)

#### Categorical Loss Function

In [5]:
def categorical_loss(y_true, y_pred):
	# input dimension is (batchsize, number of categories)
  return K.categorical_crossentropy(y_true, y_pred) # the dimension of return value is (batchsize , 1)

#### Continuous Loss Function

In [6]:
def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis) # return a tensor of shape (batch_size, 1)

# Added a postfix because now we also have poisson as a distribution
def continuous_loss_gaussian(y_true, y_pred):
	# need to return log probability for continuous gaussian loss.
	# will get a (batchsize, 6 continuous variable input) where 3 of the 6 represents mu and the others logvar
	# y_true will be (batchsize, 3)
  mu, logvar = tf.split(y_pred, num_or_size_splits = 2, axis = 1)
  return -1 * log_normal_pdf(y_true, mu, logvar) 

def continuous_loss_poisson(y_true, y_pred):
	# need to return log probability for continuous poisson loss.
	# will get a (batchsize, 6 continuous variable input) where 3 of the 6 represents mu and the others logvar
	# y_true will be (batchsize, 3)
  return tf.nn.log_poisson_loss(y_true, y_pred)

### Model Architecture
- tanh/sigmoid is used because Relu resulted in loss going to infinity
- going to delete review_log_var and review_mu since we are trying out the Poission distribution which only takes 1 parameter.

In [29]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()

import tensorflow_probability as tfp
import mdn


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions


input_shape = (original_dim,)
base_depth = 32
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(latent_dim), scale=1), reinterpreted_batch_ndims=1)

## Encoder
encoder = tfk.Sequential([
    tfkl.InputLayer(input_shape=input_shape),
    tfkl.Dense(intermediate_dim_1, activation='tanh', name='hidden_enc_1'),
    tfkl.Dense(intermediate_dim_2, activation='tanh', name='hidden_enc_2'),
    tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(latent_dim), activation=None),
    tfpl.MultivariateNormalTriL( latent_dim, activity_regularizer=tfpl.KLDivergenceRegularizer(prior)),
])


decode_1 = tfkl.Dense(intermediate_dim_2, activation='tanh', name='hidden_dec_2')
h_dec = decode_1(encoder.outputs[0])

decode_2 = tfkl.Dense(intermediate_dim_1, activation='tanh', name='hidden_dec_1')
h_dec = decode_2(h_dec)

x_pred_coordinates_mdn_layer = mdn.MDN(OUTPUT_DIMS, N_MIXES, name = 'x_pred_coordinates_mdn_layer')
x_pred_coordinates = x_pred_coordinates_mdn_layer(h_dec)

## Old coordinates layer
# x_pred_coordinates_mu_layer = tfkl.Dense(2, name='x_pred_coordinates_mu')
# x_pred_coordinates_mu = x_pred_coordinates_mu_layer(h_dec)

# x_pred_coordinates_log_var_layer = tfkl.Dense(2, name='x_pred_coordinates_log_var')
# x_pred_coordinates_log_var = x_pred_coordinates_log_var_layer(h_dec)

# x_pred_coordinates_layer = tfkl.Concatenate(axis=-1, name = 'x_pred_coordinates')
# x_pred_coordinates = x_pred_coordinates_layer([x_pred_coordinates_mu, x_pred_coordinates_log_var])

## This outputs the lambda require for poisson distribution and thats all that we need
x_pred_review_log_lambda_layer = tfkl.Dense(1,name = 'x_pred_review_log_lambda_layer')
x_pred_review = x_pred_review_log_lambda_layer(h_dec)

x_pred_binary_layer = tfkl.Dense(binary_dim, activation='sigmoid', name='x_pred_binary')
x_pred_binary = x_pred_binary_layer(h_dec) # binary cross entropy

x_pred_categorical_layer = tfkl.Dense(categorical_dim, activation='softmax', name='x_pred_categorical')
x_pred_categorical = x_pred_categorical_layer(h_dec) # categorical cross entropy

vae = tfk.Model(inputs=encoder.inputs, outputs=[x_pred_binary, x_pred_categorical, x_pred_review, x_pred_coordinates])
# vae = tfk.Model(inputs=encoder.inputs,
#                 outputs=decoder(encoder.outputs[0]))

### Compiling Model and Setting Parameters

In [35]:
optimizer = tf.keras.optimizers.Adam(lr=0.0001)
vae.compile(optimizer=optimizer, loss=[binary_loss, categorical_loss, continuous_loss_poisson, mdn.get_mixture_loss_func(OUTPUT_DIMS,N_MIXES)], loss_weights=[1, 1, 1, 1])



In [31]:
vae.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 15)]         0                                            
__________________________________________________________________________________________________
hidden_enc_1 (Dense)            (None, 50)           800         input_5[0][0]                    
__________________________________________________________________________________________________
hidden_enc_2 (Dense)            (None, 50)           2550        hidden_enc_1[0][0]               
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 14)           714         hidden_enc_2[0][0]               
____________________________________________________________________________________________

## Data Loading and Preprocessing

In [32]:
dataset = np.genfromtxt('../../datasets/yelp_business.csv',delimiter=',',skip_header=1)
dataset = dataset[~np.isnan(dataset).any(axis=1)]
test_set, train_set = np.split(dataset,[1], axis = 0)

def format_data(dataset, scaler_to_use=None ,save_scaler=True):
    # handling the categorical variables
    coordinates, ratings, reviews, is_opens = np.split(dataset, [2, 3, 4], axis = 1)
    one_hot_array = np.zeros((ratings.shape[0], categorical_dim))

    for i, r in enumerate(ratings):
        one_hot_array[i][categorical_map[r[0]]] = 1
    
    # handling coordinates
    if scaler_to_use is None:
        scaler_to_use = preprocessing.StandardScaler()
        scaler_to_use.fit(coordinates)
    
    coordinates = scaler_to_use.transform(coordinates)
    
    if save_scaler:
        dump(scaler_to_use, open('./model/standard_scaler.pkl', 'wb'))
    
    
    
#     for i, c in enumerate(coordinates):
#         coordinates[i][0] = (c[0]+180)/360
#         coordinates[i][1] = (c[1]+180)/360

    dataset = np.concatenate((coordinates, reviews, is_opens, one_hot_array), axis = 1)

    # creating the labels
    coordinates_labels = coordinates
    review_labels = reviews
    categorical_labels = one_hot_array
    binary_labels = is_opens
    return dataset, coordinates_labels, review_labels, categorical_labels, binary_labels

train_dataset, train_coordinates_labels, train_review_labels, train_categorical_labels, train_binary_labels = format_data(train_set)

## Open scaler
scaler = load(open('./model/standard_scaler.pkl', 'rb'))
test_dataset, test_coordinates_labels, test_review_labels, test_categorical_labels, test_binary_labels = format_data(test_set, scaler, False)



### Distribution of Review_Count

In [33]:
# import matplotlib.pyplot as plt
# plt.hist(train_review_labels, 1000)
# plt.show()

## Training the Model

In [34]:
vae.fit(train_dataset , [train_binary_labels, train_categorical_labels, train_review_labels, train_coordinates_labels], shuffle = True, epochs = epochs, batch_size = batch_size, validation_data=(test_dataset , [test_binary_labels, test_categorical_labels, test_review_labels, test_coordinates_labels]))

Train on 174565 samples, validate on 1 samples
Epoch 1/50
174565/174565 [==============================] - 17s 97us/sample - loss: -89.9459 - x_pred_binary_loss: 0.5043 - x_pred_categorical_loss: 2.1485 - x_pred_review_log_lambda_layer_loss: -98.5309 - x_pred_coordinates_mdn_layer_loss: 1.8392 - val_loss: -43.6719 - val_x_pred_binary_loss: 0.1467 - val_x_pred_categorical_loss: 1.4049 - val_x_pred_review_log_lambda_layer_loss: -45.8876 - val_x_pred_coordinates_mdn_layer_loss: -0.1929
Epoch 2/50
174565/174565 [==============================] - 10s 57us/sample - loss: -101.3797 - x_pred_binary_loss: 0.4547 - x_pred_categorical_loss: 2.0217 - x_pred_review_log_lambda_layer_loss: -107.9093 - x_pred_coordinates_mdn_layer_loss: 0.6540 - val_loss: -41.9807 - val_x_pred_binary_loss: 0.1930 - val_x_pred_categorical_loss: 1.4972 - val_x_pred_review_log_lambda_layer_loss: -45.4995 - val_x_pred_coordinates_mdn_layer_loss: -1.7144
Epoch 3/50
174565/174565 [==============================] - 10s 56us/

174565/174565 [==============================] - 11s 66us/sample - loss: -106.7232 - x_pred_binary_loss: 0.4384 - x_pred_categorical_loss: 1.9689 - x_pred_review_log_lambda_layer_loss: -108.4129 - x_pred_coordinates_mdn_layer_loss: -3.1339 - val_loss: -45.8126 - val_x_pred_binary_loss: 0.1638 - val_x_pred_categorical_loss: 1.4591 - val_x_pred_review_log_lambda_layer_loss: -45.9771 - val_x_pred_coordinates_mdn_layer_loss: -4.7239
Epoch 20/50
174565/174565 [==============================] - 11s 61us/sample - loss: -106.7247 - x_pred_binary_loss: 0.4384 - x_pred_categorical_loss: 1.9688 - x_pred_review_log_lambda_layer_loss: -108.3963 - x_pred_coordinates_mdn_layer_loss: -3.1375 - val_loss: -46.3860 - val_x_pred_binary_loss: 0.1589 - val_x_pred_categorical_loss: 1.4458 - val_x_pred_review_log_lambda_layer_loss: -45.5334 - val_x_pred_coordinates_mdn_layer_loss: -4.8681
Epoch 21/50
174565/174565 [==============================] - 11s 62us/sample - loss: -106.7286 - x_pred_binary_loss: 0.438

174565/174565 [==============================] - 9s 51us/sample - loss: -106.8027 - x_pred_binary_loss: 0.4384 - x_pred_categorical_loss: 1.9684 - x_pred_review_log_lambda_layer_loss: -108.3869 - x_pred_coordinates_mdn_layer_loss: -3.1813 - val_loss: -45.8910 - val_x_pred_binary_loss: 0.1714 - val_x_pred_categorical_loss: 1.5921 - val_x_pred_review_log_lambda_layer_loss: -44.0058 - val_x_pred_coordinates_mdn_layer_loss: -4.6647
Epoch 38/50
174565/174565 [==============================] - 11s 61us/sample - loss: -106.8585 - x_pred_binary_loss: 0.4383 - x_pred_categorical_loss: 1.9687 - x_pred_review_log_lambda_layer_loss: -108.4267 - x_pred_coordinates_mdn_layer_loss: -3.1825 - val_loss: -46.2391 - val_x_pred_binary_loss: 0.1692 - val_x_pred_categorical_loss: 1.4902 - val_x_pred_review_log_lambda_layer_loss: -45.9453 - val_x_pred_coordinates_mdn_layer_loss: -5.2102
Epoch 39/50
174565/174565 [==============================] - 15s 86us/sample - loss: -106.9082 - x_pred_binary_loss: 0.4383

## Building the Decoder

In [36]:
decode_input = Input(shape=(latent_dim, ), name = 'decode_input')
decode_layer_1 = decode_1(decode_input)
decode_layer_2 = decode_2(decode_layer_1)

decode_x_pred_coordinates=x_pred_coordinates_mdn_layer(decode_layer_2)
# x_pred_coordinates_mu = x_pred_coordinates_mu_layer(decode_layer_2)
# x_pred_coordinates_log_var = x_pred_coordinates_log_var_layer(decode_layer_2)
# decode_x_pred_coordinates = x_pred_coordinates_layer([x_pred_coordinates_mu, x_pred_coordinates_log_var])

decode_x_pred_review = x_pred_review_log_lambda_layer(decode_layer_2)

decode_x_pred_binary = x_pred_binary_layer(decode_layer_2) # binary cross entropy
decode_x_pred_categorical = x_pred_categorical_layer(decode_layer_2) # categorical cross entropy


decoder = Model(decode_input, [decode_x_pred_coordinates, decode_x_pred_review, decode_x_pred_categorical, decode_x_pred_binary])

In [37]:
SVG(model_to_dot(decoder, show_shapes=True)
    .create(prog='dot', format='svg'))

TypeError: 'InputLayer' object is not iterable

### Saving the Models and Metadata
#### Saving Models (Actually, only the decoder matter)

In [38]:
model_json = vae.to_json()
with open("./model/vae_full_model.json", "w") as json_file:
    json_file.write(model_json)
vae.save_weights("./model/vae_full_model.h5")
print("Saved model to disk")

encoder = Model(x, [z_mu, z_log_var])
model_json = encoder.to_json()
with open("./model/vae_encoder.json", "w") as json_file:
    json_file.write(model_json)
encoder.save_weights("./model/vae_encoder.h5")
print("Saved model to disk")

model_json = decoder.to_json()
with open("./model/vae_decoder.json", "w") as json_file:
    json_file.write(model_json)
decoder.save_weights("./model/vae_decoder.h5")
print("Saved model to disk")


Saved model to disk
Saved model to disk
Saved model to disk


#### Generating and Saving Metadata

In [39]:
vae_sample = train_dataset[np.random.choice(len(train_dataset), size=10000, replace=False)]
vae_sample_mu, vae_sample_log_var = encoder.predict(vae_sample, batch_size=batch_size)
np.save('./model/sample_mu.npy', vae_sample_mu)
np.save('./model/sample_log_var.npy', vae_sample_log_var)

### Generating some Samples for Testing
#### Functions for data generation

In [46]:
reverse_categorical_map = {0:0, 1:0.5, 2:1, 3:1.5, 4:2, 5:2.5, 6:3, 7:3.5, 8:4, 9:4.5, 10:5}
def sample(model, input_mu, input_log_var, samples_per_z=1):
    multiplied_input_mu = np.repeat(input_mu, samples_per_z, axis=0)
    multiplied_input_log_var = np.repeat(input_log_var, samples_per_z, axis=0)
    eps = np.random.normal(size=(multiplied_input_mu.shape[0], latent_dim))
    z = reparameterize(multiplied_input_mu, multiplied_input_log_var, eps)
    predictions = model.predict(z, batch_size = None, steps = 1)
    return reconstruct(predictions)
    
def reconstruct(predictions):
    coordinates, review, categorical, binary = predictions
    
    # coordinates handled here
    print(coordinates.shape)
    coordinates_data = np.apply_along_axis(mdn.sample_from_output, 1, coordinates, OUTPUT_DIMS, N_MIXES, temp=1.0)
    coordinates_data = np.squeeze(coordinates_data, axis=1)
#     mu, log_var = np.split(coordinates, indices_or_sections = 2,axis = 1)
#     eps = np.random.normal(size=mu.shape)
#     coordinates_data = reparameterize(mu, log_var, eps)
    scaler = load(open('./model/standard_scaler.pkl', 'rb'))
    coordinates_data = scaler.inverse_transform(coordinates_data)
    print(coordinates_data.shape)
    
    for i, c in enumerate(coordinates_data):
        if c[0] > 180.0:
            coordinates_data[i][0]= 180.0
        if c[0] < -180.0:
            coordinates_data[i][0]= -180.0
        if c[1] > 180.0:
            coordinates_data[i][1]= 180.0
        if c[1] < -180.0:
            coordinates_data[i][1]= -180.0
    
    ## review_count handled here
    exp_log_review = np.exp(review)
    review_data = np.random.poisson(lam=exp_log_review, size = review.shape)
    for i, r in enumerate(review_data):
        if r[0] < 0:
            review_data[i][0] = 0
        review_data[i][0] = float(int(review_data[i][0]))
    
    categorical = np.apply_along_axis(lambda t : np.random.multinomial(1,t), -1, categorical)
    categorical = np.apply_along_axis(lambda t : np.argmax(t), -1, categorical)
    categorical = np.expand_dims(categorical, axis = -1)
    categorical_data = np.apply_along_axis(lambda t : float(reverse_categorical_map[t[0]]), -1, categorical)
    categorical_data = np.expand_dims(categorical_data, axis = -1)
    binary_data = np.apply_along_axis(lambda t: np.random.binomial(1, t), -1, binary)
#     coordinates, reviews = np.split(continuous_data, indices_or_sections=[2], axis = 1)
    return np.concatenate([coordinates_data, categorical_data, review_data, binary_data], axis = 1)
    

def reparameterize(input_mu, input_log_var, eps):
    sigma = np.exp(0.5*input_log_var)
    return eps*sigma + input_mu

### Generating Samples
Make use of the funciton sample to generate samples with our model. U need to supply an array of mu and their respective log var in a separate array to do that. 

In [47]:
vae_samples = sample(decoder, vae_sample_mu, vae_sample_log_var)

# Saving the samples in a separate file
file_name = './samples/vae_8_sample_' + str(len(vae_samples)) + '.csv'
np.savetxt(file_name, vae_samples, delimiter = ',', header='latitude,longitude,stars,review_count,is_open')

(10000, 50)
(10000, 2)


In [48]:
vae_samples[1:10]

array([[  36.12722656, -114.97610985,    1.        ,    0.        ,
           1.        ],
       [  43.64995768,  -79.71934736,    4.        ,    2.        ,
           1.        ],
       [  41.24716502,  -82.14576402,    3.        ,    3.        ,
           1.        ],
       [  38.83486706,  -81.44808577,    1.        ,    1.        ,
           1.        ],
       [  33.52494004, -112.20338825,    5.        ,    6.        ,
           1.        ],
       [  43.83154703,  -79.53044245,    3.        ,    4.        ,
           1.        ],
       [  33.47541187, -111.71694681,    5.        ,    3.        ,
           1.        ],
       [  43.03446893,  -80.8353163 ,    1.5       ,    3.        ,
           1.        ],
       [  45.55855775,  -73.54997901,    2.        ,    7.        ,
           1.        ]])